In [1]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.INFO)
from rexpand_pyutils_file import write_file

from utils.conversation_data import load_conversation_data
from utils.json import to_json_compatible
from models.context import Context
from nodes.orchestrator import orchestrate
from models.workflow import State


INFO:root:Read JSON data from ./input/categories.json
INFO:root:Read JSON data from ./input/categories.json


In [2]:
# Load all conversations
conversations = load_conversation_data('./input/convo_2454_rows.xlsx')


INFO:root:Read sheet 'Result 1' from Excel file ./input/convo_2454_rows.xlsx
INFO:root:Read EXCEL data from ./input/convo_2454_rows.xlsx with 2454 rows


In [3]:
# Select a conversation to process
index = 2
messages_end = 5
messages = conversations[index][:messages_end]
context = Context(messages=messages)
write_file('./output/current_messages.json', to_json_compatible(messages))


INFO:root:Wrote JSON data to ./output/current_messages.json


In [4]:
# Fresh start. In this case, no human in the loop with topics suggested
state = State(context=context)
state = orchestrate(state)

print(state.step)
print(state.classified_category)
print(state.suggested_topics)
print(state.selected_topics)
print(state.generated_reply_message)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:Wrote JSON data to ./.cache/33c19d20ca9ffb7a2db57028abbbe12e.json
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:Wrote JSON data to ./.cache/465fe4f8b01701cea98b0a7f2d7d388e.json


next: select topics
ClassifierResult(category='no_reply', confidence=0.9, reason="The job seeker has just replied to the referrer's request for more information about the referral by providing the job link and further explanation, but the referrer has not responded yet.", referenced_message_ids=['fcf62787-5474-4ba4-a04d-0f7c5f9e9a5c'])
TopicSuggesterResult(topics=[Topic(topic='Thank you', confidence=0.9, reason="The job seeker has expressed gratitude for the referrer's reply, which is polite and appropriate at this stage."), Topic(topic='Express interest', confidence=0.95, reason='The job seeker shows clear interest in the Junior Data Analyst position at EY and wants to pursue the opportunity.'), Topic(topic='Express background match', confidence=0.6, reason='Though the job seeker has not explicitly described their qualifications, implying interest and asking for referral suggests a potential match in background.'), Topic(topic='Ask for a brief call', confidence=0.7, reason='The job se

In [5]:
# Continue with selecting all suggested topics to generate a reply
state.selected_topics = state.suggested_topics
state = orchestrate(state)

print(state.step)
print(state.classified_category)
print(state.suggested_topics)
print(state.selected_topics)
print(state.generated_reply_message)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:root:Wrote JSON data to ./.cache/8279752a271e0fc52e61d9a8a8b10582.json


end: reply generated
ClassifierResult(category='no_reply', confidence=0.9, reason="The job seeker has just replied to the referrer's request for more information about the referral by providing the job link and further explanation, but the referrer has not responded yet.", referenced_message_ids=['fcf62787-5474-4ba4-a04d-0f7c5f9e9a5c'])
TopicSuggesterResult(topics=[Topic(topic='Thank you', confidence=0.9, reason="The job seeker has expressed gratitude for the referrer's reply, which is polite and appropriate at this stage."), Topic(topic='Express interest', confidence=0.95, reason='The job seeker shows clear interest in the Junior Data Analyst position at EY and wants to pursue the opportunity.'), Topic(topic='Express background match', confidence=0.6, reason='Though the job seeker has not explicitly described their qualifications, implying interest and asking for referral suggests a potential match in background.'), Topic(topic='Ask for a brief call', confidence=0.7, reason='The job s